<a href="https://colab.research.google.com/github/GiovanniSorice/Deep_Music_Generator/blob/main/notebooks/Music_Generation_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer Music Generator 



In this notebook, we use an Transformer to generate some music.


**This notebook was inspired (and part of the code comes from it) by [Music_Generation_LSTM](https://colab.research.google.com/drive/19TQqekOlnOSW36VCL8CPVEQKBBukmaEQ#scrollTo=DDOBVWULXfpz)**




**Load dependencies**

In [1]:
pip install compressive_transformer_pytorch

In [2]:
import torch
import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from compressive_transformer_pytorch import CompressiveTransformer
from compressive_transformer_pytorch import AutoregressiveWrapper
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset
from tensorflow.keras import utils
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt
import glob
import pickle
from music21 import converter, instrument, stream, note, chord
import math
import shutil

In [3]:
# Set to false if you are not running
# this notebook in Google Colaboratory
run_on_colab = True

**Set hyperparameters**

In [4]:
# output directory name:
output_dir = '/content/drive/My Drive/ISPR_project/Transformer/'
current_path ='/content/drive/My Drive/ISPR_project/'
# training:
epochs = 2000
batch_size = 64
learning_rate=1e-4
# vector-space embedding: 
n_dim = 64 
sequence_length = 64


VALIDATE_EVERY  = 5

GENERATE_EVERY  = 500



**Save model function**

In [5]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, output_dir+filename)
    if is_best:
        shutil.copyfile(output_dir+filename, output_dir+'model_best.pth.tar')

**Google drive configuration (only Colab)**

In [6]:
if(run_on_colab):
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

Mounted at /content/drive


**Load data** \\
Original MIDI files
 I have obtained  **MIDI files** from [The Lakh MIDI Dataset v0.1](https://colinraffel.com/projects/lmd/). 

## Processing data

Let's process the files, and load them into **music21**

In [9]:
file = current_path+"midi_songs/dataset/Metal/Metallica/Am I Evil?.mid"
midi = converter.parse(file)
notes_to_parse = midi.flat.notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.chord.Chord E2 E3 B3 E4> 0.0
<music21.chord.Chord E2 E3 B3 E4> 0.0
<music21.note.Note E> 0.0
<music21.chord.Chord C2 C#3> 0.0
<music21.note.Note G#> 2.0
<music21.chord.Chord D3 A3 D4> 3.0
<music21.chord.Chord D3 A3 D4> 3.0
<music21.note.Note D> 3.0
<music21.chord.Chord C#3 C2> 3.0
<music21.chord.Chord B3 E3 E4> 3.5


I will process all MIDI files obtaining data from each note of chord.

- If I process a **note**, I will store in the list a string representing the pitch (the note name) and the octave.

- If I process a **chord** (Remember that chords are set of notes that are played at the same time) I will store a different type of string with numbers separated by dots. Each number represents the pitch of a chord note. 

As you can see, **I are not considering yet time offsets of each element**. In this first version, we won't consider them, so all the notes and chords will have the same duration. Maybe, in the future, I will consider them.

I are creating a big list with all the elements of all the compositions.

In [10]:
notes_for_instruments = []
for i,file in enumerate(glob.glob(current_path+"midi_songs/dataset/*/*/*.mid")):
      midi = converter.parse(file)
      print('Parsing file ', i, " ", file)
      notes_to_parse = None
      try:  # file has instrument parts
          s2 = instrument.partitionByInstrument(midi)
          notes_to_parse = s2.recurse()
      except:  # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      notes_instrument = []
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes_instrument.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes_instrument.append('.'.join(str(n) for n in element.normalOrder))
      notes_for_instruments.append(notes_instrument)
with open(current_path + 'notes_for_instruments', 'wb') as filepath:
    pickle.dump(notes_for_instruments, filepath)


Parsing file  0   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/All God's People.mid
Parsing file  1   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/All God's People.1.mid
Parsing file  2   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/All God's People.2.mid
Parsing file  3   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/A Kind of Magic.1.mid
Parsing file  4   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/A Kind of Magic.3.mid
Parsing file  5   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/A Kind of Magic.2.mid
Parsing file  6   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/Bohemian Rhapsody.1.mid
Parsing file  7   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/Another One Bites The Dust.1.mid
Parsing file  8   /content/drive/My Drive/ISPR_project/midi_songs/dataset/Rock/Queen/Another One Bites The Dust.mid
Parsing file  

In [11]:
notes_for_instruments_validation = []
for i,file in enumerate(glob.glob(current_path+"midi_songs/validation/*.mid")):
      midi = converter.parse(file)
      print('Parsing file ', i, " ", file)
      notes_to_parse = None
      try:  # file has instrument parts
          s2 = instrument.partitionByInstrument(midi)
          notes_to_parse = s2.recurse()
      except:  # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      notes_instrument = []
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes_instrument.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes_instrument.append('.'.join(str(n) for n in element.normalOrder))
      notes_for_instruments_validation.append(notes_instrument)
with open(current_path + 'VALIDATION_notes_for_instruments', 'wb') as filepath:
    pickle.dump(notes_for_instruments_validation, filepath)


Parsing file  0   /content/drive/My Drive/ISPR_project/midi_songs/validation/Crazy Little Thing Called Love.mid
Parsing file  1   /content/drive/My Drive/ISPR_project/midi_songs/validation/Nothing Else Matters.2.mid
Parsing file  2   /content/drive/My Drive/ISPR_project/midi_songs/validation/King Nothing.1.mid
Parsing file  3   /content/drive/My Drive/ISPR_project/midi_songs/validation/Fixxxer.mid
Parsing file  4   /content/drive/My Drive/ISPR_project/midi_songs/validation/Motorbreath.mid
Parsing file  5   /content/drive/My Drive/ISPR_project/midi_songs/validation/Porch.mid
Parsing file  6   /content/drive/My Drive/ISPR_project/midi_songs/validation/A Kind of Magic.mid
Parsing file  7   /content/drive/My Drive/ISPR_project/midi_songs/validation/Don't Chain My Heart.mid
Parsing file  8   /content/drive/My Drive/ISPR_project/midi_songs/validation/Se tornerai.1.mid
Parsing file  9   /content/drive/My Drive/ISPR_project/midi_songs/validation/Pamela.1.mid


In [7]:
with open(current_path + 'notes_for_instruments', 'rb') as f:
    notes_for_instruments = pickle.load(f)

In [8]:
with open(current_path + 'VALIDATION_notes_for_instruments', 'rb') as f:
    notes_for_instruments_validation = pickle.load(f)

I obtain the number of different notes in our dataset, because this will be the **number of possible output classes**  of our model.

In [9]:
# Count different possible outputs
n_vocab = (len(set(item for notes_for_instrument in notes_for_instruments for item in notes_for_instrument)))
n_vocab

833

In [10]:
# Count different possible outputs valifation
print(len(set(item for notes_for_instrument in notes_for_instruments_validation for item in notes_for_instrument)))

229


**Preprocess data** \\
Now, there is some **data processing** that I have to do:

- I will map each pitch or chord to an integer
- I will create pairs of input sequences and its corresponding output note

I can try different **sequence_length** to obtain different results. In this first version, I will use a sequence_length of 100.

The network will made its prediction of the next note (or chord), based on the previous *sequence_length* notes (or chords). 


In [11]:
# get all pitch names
pitchnames_training = set(item for notes_for_instrument in notes_for_instruments for item in notes_for_instrument)
pitchnames_validation = set(item for notes_for_instrument in notes_for_instruments_validation for item in notes_for_instrument)
pitchnames = sorted(pitchnames_training.union(pitchnames_validation))

In [12]:
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
for notes in notes_for_instruments:
    if len(notes) - sequence_length<=0:
        print("canzone troppo corta")
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
      # Map pitches of sequence_in to integers
      network_input.append([note_to_int[char] for char in notes[i:i + sequence_length]])
n_patterns = len(network_input)
# reshape the input into a format compatible with Transormer layers
network_input = np.reshape(network_input, (n_patterns, sequence_length))

In [13]:
# create a dictionary to map pitches to integers
note_to_int_validation = dict((notes_validation, number) for number, notes_validation in enumerate(pitchnames))
network_input_validation = []
network_output_validation = []
for notes_validation in notes_for_instruments_validation:
    if len(notes_validation) - sequence_length<=0:
        print("canzone troppo corta")
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes_validation) - sequence_length, 1):
      # Map pitches of sequence_in to integers
      network_input_validation.append([note_to_int_validation[char] for char in notes_validation[i:i + sequence_length]])
n_patterns_validation = len(network_input_validation)
# reshape the input into a format compatible with Transormer layers
network_input_validation = np.reshape(network_input_validation, (n_patterns_validation, sequence_length))

Let's see the new metwork_input size

In [14]:
network_input.shape

(366889, 64)

In [15]:
network_input_validation.shape

(36341, 64)

**Design neural network architecture** 

In [16]:
def create_network(sequence_length, n_vocab):
    """ create the structure of the neural network """
    model = CompressiveTransformer(
    num_tokens = n_vocab,
    dim = sequence_length,
    depth = 6,
    seq_len = sequence_length,
    mem_len = sequence_length,
    cmem_len = 256,
    cmem_ratio = 4,
    memory_layers = [5,6],
    gru_gated_residual = False
    )

    model = AutoregressiveWrapper(model)
    model.cuda()
    return model

In [17]:
model = create_network(sequence_length,n_vocab)

print(model)


AutoregressiveWrapper(
  (net): CompressiveTransformer(
    (token_emb): Embedding(833, 64)
    (to_model_dim): Identity()
    (to_logits): Sequential(
      (0): Identity()
      (1): Linear(in_features=64, out_features=833, bias=True)
    )
    (attn_layers): ModuleList(
      (0): Residual(
        (fn): PreNorm(
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (fn): SelfAttention(
            (compress_mem_fn): ConvCompress(
              (conv): Conv1d(64, 64, kernel_size=(4,), stride=(4,))
            )
            (to_q): Linear(in_features=64, out_features=64, bias=False)
            (to_kv): Linear(in_features=64, out_features=128, bias=False)
            (to_out): Linear(in_features=64, out_features=64, bias=True)
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (dropout): Dropout(p=0.0, inplace=False)
            (reconstruction_attn_dropout): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (1): Residual(

In [18]:
def cycle(loader):
    while True:
        for data in loader:
          yield data


data_train = torch.from_numpy(network_input).cuda()
train_loader = torch.utils.data.DataLoader(data_train, batch_size=32) 
cycle_train_loader  = cycle(DataLoader(data_train, batch_size = data_train.shape[0]))
num_batches=math.ceil(data_train.shape[0]/batch_size) # Total number of batches

In [19]:
#Validation
data_validation = torch.from_numpy(network_input_validation).cuda()
validation_loader = torch.utils.data.DataLoader(data_validation, batch_size=32) 
cycle_validation_loader  = cycle(DataLoader(data_validation, batch_size = data_validation.shape[0]))
num_batches_val=math.ceil(data_validation.shape[0]/batch_size) # Total number of batches

In [20]:
# optimizer

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In case we want to use previously trained weights, to continue the training in the point we left it, we should load them into the model.

This is very useful in Google Colaboratory, that usually kills the virtual machine that is executing the Jupyter notework after a certime amount of time. If this happens to you, you should have to look for the last weights file in your configured Drive account and use it to train the network.


In [ ]:
# In case we want to use previously trained weights
weights = "model_best.pth.tar"
checkpoint = torch.load("/content/drive/MyDrive/ISPR_project/Transformer/model_best.pth.tar")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [ ]:
# training

for i in tqdm.tqdm(range(epochs), mininterval=20., desc='training'):
    model.train()
    tot_loss = 0.0
    is_best=0
    best_loss_value=n_vocab
    avg_loss_val=0
    for mlm_loss, aux_loss, is_last in model(next(cycle_train_loader), max_batch_size = batch_size, return_loss = True):
        loss = mlm_loss + aux_loss

        loss.backward()

        tot_loss+=loss;

        if is_last:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            optimizer.zero_grad()
    
    if i % VALIDATE_EVERY == 0 or i==epochs-1:
      model.eval()
      with torch.no_grad():
          for loss_val, aux_loss_val, is_last_val in model(next(cycle_validation_loader), max_batch_size = batch_size, return_loss = True):
            avg_loss_val+=loss_val/num_batches_val;

            if is_last_val:
              print(f'\n validation loss: {avg_loss_val.item():.4f}')


    avg_loss=tot_loss/num_batches

    if i%5==0 or i==epochs-1:
      if best_loss_value>avg_loss:
        best_loss_value=avg_loss;
        is_best=1

      save_checkpoint({
      'epoch': i,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict' : optimizer.state_dict(),
      'loss':avg_loss.item(),
     }, is_best, 'Tran_64_Checkpoint'+str(i)+'_'+"{:.4f}".format(avg_loss.item())+'.pth.tar')
      is_best=0
    print(f'\n Epoch: {i} |Training loss: {avg_loss.item():.4f}')
print('\nTraining complete.')






**Music generation**

In [ ]:
# In case we want to use previously trained weights
weights = "model_best.pth.tar"
checkpoint = torch.load(output_dir+weights)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [ ]:
# Generate network input again
network_input = []
network_output = []
for i in range(0, len(notes) - sequence_length, 1):
  network_input.append([note_to_int[char] for char in notes[i:i + sequence_length]])
n_patterns = len(network_input)
network_input = np.reshape(network_input, (n_patterns, sequence_length))


The workflow now is:


1.   Pick a **seed sequence** randomly from your list of inputs (*pattern* variable)
2.   Pass it as input for your model to generate a new element (note or chord)
3.   Add the new element to your final song and to your *pattern* list
4.   Remove the first item from *pattern*
5.   Go to step 2


In [ ]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = torch.from_numpy(network_input[start]).cuda()

prediction_output = model.generate(pattern, 500)


In [ ]:
result_sample=[]

for i in range(500):
  print(i)
  result = int_to_note[prediction_output[i].item()]
  print('\r', 'Predicted ', i, " ",result, end='')
  result_sample.append(result)

prediction_output=result_sample

0
 Predicted  0   61
 Predicted  1   4.62
 Predicted  2   6.113
 Predicted  3   64
 Predicted  4   6.115
 Predicted  5   A46
 Predicted  6   4.67
 Predicted  7   F48
 Predicted  8   69
 Predicted  9   610
 Predicted  10   5.7.9.011
 Predicted  11   2.3.7.1012
 Predicted  12   D513
 Predicted  13   C514
 Predicted  14   5.7.9.015
 Predicted  15   C516
 Predicted  16   4.617
 Predicted  17   B-118
 Predicted  18   10.2.519
 Predicted  19   C520
 Predicted  20   6.1121
 Predicted  21   622
 Predicted  22   F223
 Predicted  23   6.1124
 Predicted  24   4.625
 Predicted  25   B-226
 Predicted  26   B-127
 Predicted  27   A428
 Predicted  28   629
 Predicted  29   C530
 Predicted  30   E-331
 Predicted  31   F232
 Predicted  32   4.633
 Predicted  33   534
 Predicted  34   5.1035
 Predicted  35   4.636
 Predicted  36   637
 Predicted  37   4.638
 Predicted  38   4.639
 Predicted  39   F240
 Predicted  40   4.641
 Predicted  41   B-242
 Predicted  42   643
 Predicted  43   5.7.9.044
 Predicte

The last step is creating a MIDI file from the predictions.

**music21** will help us again for this task. We should create a **Stream** and add to it the predicted notes and chords.

We are adding an offset of 0.5 between elements.

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'